In [9]:
# =========================================================
# 🚀 Grover's Search Algorithm – Qiskit 2.x Enhanced Version
# =========================================================

# STEP 1 — Install dependencies
!pip install qiskit qiskit-aer matplotlib pylatexenc --quiet

# STEP 2 — Imports
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator, noise
import matplotlib.pyplot as plt
import math

# =========================================================
# STEP 3 — ORACLE & DIFFUSER
# =========================================================
def oracle(qc, n, marked_state):
    """Applies the oracle that marks the target state."""
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

def diffuser(qc, n):
    """Implements the inversion-about-the-mean diffuser."""
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))

# =========================================================
# STEP 4 — BUILD GROVER CIRCUIT
# =========================================================
def grover_search(marked_state, iterations=1):
    """Constructs Grover’s search circuit for a given marked state."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    for _ in range(iterations):
        oracle(qc, n, marked_state)
        diffuser(qc, n)
    qc.measure(range(n), range(n))
    return qc

# =========================================================
# STEP 5 — RUNNER (WITH NOISE OPTION)
# =========================================================
def run_grover(qc, use_noise=False):
    """Runs the Grover circuit and plots histogram results."""
    simulator = AerSimulator()
    if use_noise:
        # Create a simple depolarizing noise model
        noise_model = noise.NoiseModel()
        error_1 = noise.depolarizing_error(0.001, 1)
        error_2 = noise.depolarizing_error(0.01, 2)
        noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
        noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
        compiled = transpile(qc, simulator)
        result = simulator.run(compiled, shots=1024, noise_model=noise_model).result()
        print("🧩 Running with noise model...")
    else:
        compiled = transpile(qc, simulator)
        result = simulator.run(compiled, shots=1024).result()
        print("🧠 Running ideal (noise-free) simulation...")

    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    return counts

# =========================================================
# STEP 6 — MAIN EXECUTION
# =========================================================
if __name__ == "__main__":
    # --- USER CONFIGURABLE PARAMETERS ---
    marked_state = "101"   # Change this → try "001", "010", "111", "1100", etc.
    iterations = 2         # Increase to 2 or 3 for better amplification
    use_noise = False      # Set True to compare noisy vs. ideal simulator

    print(f"🔍 Searching for marked state: {marked_state}")
    qc = grover_search(marked_state, iterations)

    # --- Circuit Visualization ---
    try:
        qc.draw('mpl')     # Pretty diagram if pylatexenc is available
    except Exception:
        print("⚠️ pylatexenc not found — showing text view instead.")
        print(qc.draw('text'))

    # --- Run Simulation ---
    counts = run_grover(qc, use_noise=use_noise)

    # --- Theoretical optimal iteration count ---
    n = len(marked_state)
    N = 2 ** n
    opt_iters = round((math.pi / 4) * math.sqrt(N))
    print(f"📘 Suggested optimal iterations ≈ {opt_iters} for {N} states.")

# =========================================================
# STEP 7 — (OPTIONAL) REAL HARDWARE EXECUTION
# =========================================================
# Uncomment this section if you have IBM Quantum credentials set up
#
# from qiskit_ibm_provider import IBMProvider
# provider = IBMProvider()
# backend = provider.get_backend('ibmq_qasm_simulator')  # or a real backend like 'ibm_osaka'
# compiled = transpile(qc, backend)
# job = backend.run(compiled, shots=1024)
# from qiskit.tools.monitor import job_monitor
# job_monitor(job)
# result = job.result()
# plot_histogram(result.get_counts())


🔍 Searching for marked state: 101
⚠️ pylatexenc not found — showing text view instead.
     ┌───┐          ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐»
q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├»
     ├───┤┌───┐  │  ├───┤├───┤┌───┐       │  ├───┤├───┤┌───┐       │  ├───┤»
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├»
     ├───┤├───┤┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤»
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
c: 3/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐               ┌───┐┌───┐     ┌─┐      
«q_0: ┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
«     ├───┤┌───┐       │  ├───┤├───┤     └╥┘┌─┐   
«q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
«     ├───┤├───┤